In [1]:
import pandas as pd
import datetime as dt
import glob
pd.options.mode.chained_assignment = None

In [2]:
#The code that makes state level csv or parquet files from the orginal file.
#Only uncomment if you want to do this, it takes a long time to run

'''
df = pd.read_csv('POUS_export_citybyutility_raw_us_west-2.txt')

#txt file adds extra column and '/t' to the front of headers - drop those
df.drop(columns={'Unnamed: 8'}, inplace=True)
df.columns = df.columns.str.strip('\t')

#if you want in a different directory add Path + '/'
for name, group in grouped:
    group.to_csv("{}.csv".format(name))
    print(len(group))
'''

'\ndf = pd.read_csv(\'POUS_export_citybyutility_raw_us_west-2.txt\')\n\n#txt file adds extra column and \'/t\' to the front of headers - drop those\ndf.drop(columns={\'Unnamed: 8\'}, inplace=True)\ndf.columns = df.columns.str.strip(\'\t\')\n\n#if you want in a different directory add Path + \'/\'\nfor name, group in grouped:\n    group.to_csv("{}.csv".format(name))\n    print(len(group))\n'

In [3]:
df_Ca = pd.read_parquet('California.parquet')

In [4]:
df_Ca['RecordedDateTimet'] = pd.to_datetime(df_Ca.index)
df_Ca['RecordedDateTime_dt'] = pd.to_datetime(df_Ca['RecordedDateTimet'])

df_Ca.sort_values(by='RecordedDateTimet', inplace=True)
df_Ca.reset_index(inplace=True)
df_Ca['RecordedDateTime'] =df_Ca['RecordedDateTime'].dt.strftime("%Y-%m-%d, %H:%M:%S")


## Assumption based on CA data. 

Most of the other states just have the city header as blank (NaN) in the raw data. 

However, for CA there are 1557 unique city labels (this includes the 'unknown' label) and there are 67 counties.

15 counties contain an unknown label.

Here I do groupby County and City, where 'unknown' cities within each county are grouped together, but cities that are labeled are also grouped together. I think this makes the labeled cities more accurate, but there is no way to tell about the 'unknown' labels. 

For instance, Berekley and Oakland are both labeled cities in the same county, if I leave out grouping by the city, there are times an outage in Berekley is calculated with an outage in Oakland - truely at the county level, but somewhat misleading.However, if we think this treats the 'unknown' city values unfairly I can change this.


In [5]:
df = df_Ca.groupby([ 'CountyFIPS', 'CityName'])

In [6]:
state = []
#make the series we want to add 
df_Ca['Blackout_Duration'] = ''
df_Ca['Blackout_Start'] = ''
df_Ca['Blackout_End'] = ''

'''
where name is tuple (county,city) and group is all data per county, city
this calculates an outage for each change in CustomersOut
'''
for name, group in df:
    
    #if data stops in the middle of an outage drop the data after the last zero
    group= group[group.index <= group.where(group['CustomersOut'] ==0).last_valid_index()] 
    
    for i in range(0, len(group)):
        #counter
        j = i + 1
        
        #keep counting 
        if j < len(group):

            while group['CustomersOut'].iloc[j] != 0:

                j = j + 1
                
            #this would mean we have reached the end of the data - prob not needed but being safe.
            if j >= len(group):

                break
            #we have reached a zero and can calculate duration for that outage, i = inital, j=counter
            else:

                group['Blackout_Duration'].iloc[i] =  group['RecordedDateTime_dt'].iloc[j] - group['RecordedDateTime_dt'].iloc[i]
                group['Blackout_Start'].iloc[i] =  group['RecordedDateTime_dt'].iloc[i]
                group['Blackout_End'].iloc[i] =  group['RecordedDateTime_dt'].iloc[j]

                
                
           
    group.drop(columns=['RecordedDateTimet', 'RecordedDateTime_dt', 'RecordedDateTime'], inplace=True)
    state.append(group)
    #could also do this here
    #group.to_csv("{}.csv".format(name), index=False)
final = pd.concat(state, axis=0, ignore_index=True)
df_final =final[final['CustomersOut']!=0].dropna()   
    

In [7]:
df_final 

,UtilityName,StateName,CountyName,CityName,CountyFIPS,CustomersTracked,CustomersOut,Blackout_Duration,Blackout_Start,Blackout_End
0,Pacific Gas and Electric Company,California,Alameda,Albany,6001.0,1,1,0 days 01:09:00,2017-10-13 17:33:00,2017-10-13 18:42:00
2,Pacific Gas and Electric Company,California,Alameda,Albany,6001.0,1,1,0 days 02:29:00,2017-10-15 13:33:00,2017-10-15 16:02:00
4,Pacific Gas and Electric Company,California,Alameda,Albany,6001.0,1,1,0 days 02:00:00,2017-10-19 17:43:00,2017-10-19 19:43:00
6,Pacific Gas and Electric Company,California,Alameda,Albany,6001.0,1,1,0 days 03:40:00,2017-10-20 02:22:00,2017-10-20 06:02:00
8,Pacific Gas and Electric Company,California,Alameda,Albany,6001.0,36,36,0 days 02:09:00,2017-10-20 07:43:00,2017-10-20 09:52:00
...,...,...,...,...,...,...,...,...,...,...
2001999,Pacific Gas and Electric Company,California,Yuba,Wheatland,6115.0,1540,139,0 days 00:00:00,2021-06-17 00:28:53,2021-06-17 00:28:53
2002001,Pacific Gas and Electric Company,California,Yuba,Wheatland,6115.0,1540,389,0 days 02:00:17,2021-06-21 17:30:44,2021-06-21 19:31:01
2002002,Pacific Gas and Electric Company,California,Yuba,Wheatland,6115.0,1540,1,0 days 01:40:28,2021-06-21 17:50:33,2021-06-21 19:31:01
2002004,Pacific Gas and Electric Company,California,Yuba,Wheatland,6115.0,1540,34,0 days 06:00:45,2021-06-28 16:29:55,2021-06-28 22:30:40


In [8]:
#using a parquet file because data is dropped when you open in .csv
#this is in the data file on github
df_final.to_parquet('California_OutageDuration.parquet', index=False)

## Question
It seems that the 'CustomersTracked' changes based on some outage - see Alameda County and Albany City - thus, this may impact how the calculate IEEE metrics? SAIDI, etc..?